In [13]:
import pandas as pd
import os
import glob
import time
file_list = []
file_list_lab = []
date = "2019-01-10"
input_dir = "/scratch/rk9cx/conn_log_labelled_runtime/"+date+"/"
output_dir = "/scratch/rk9cx/conn_log_labelled_runtime/unknown_"+date+"/"
for file in glob.glob(os.path.join(input_dir, 'anon.conn_tcp.*.log')):
    file_list.append(file.split(date+"/")[1])
for file in glob.glob(os.path.join(input_dir, 'anon.conn_tcp.*.csv')):
    file_list_lab.append(file.split(date+"/")[1])

In [14]:
def f(row):
    if (row['honeypot'] == 1 or row['blacklist'] == 1) and row['whitelist'] != 1 :
        val = 1
    elif row['whitelist'] == 1 and (row['honeypot'] != 1 and row['blacklist'] != 1):
        val = 0
    else:
        val = -1
    return val

In [15]:
len(file_list_lab)

24

In [ ]:
rows = 0
for file in file_list[0:8]:
    print("partition")
    start = time.time()
    labelled_logs = pd.read_csv(input_dir+file.strip(".log")+"_LABELS.csv",
                                usecols=[2,3,4,5])
    labelled_logs.columns = ["src_ip_ext","blacklist","honeypot","whitelist"]
    logs = pd.read_csv(input_dir+file, skiprows=8, 
                       usecols=[0,2,3,4,5,6,7,8,9,10,11,12,14],
                       header = None, delimiter ="\t")
    logs = logs.rename({0:'ts', 2:"src_ip", 3: "src_port", 4:'dest_ip',
                   5:'dest_port', 6:'duration', 7:'src_bytes',
                   8:'dest_bytes', 9:'conn_state',10:'history',
                   11:'src_pkts', 12:'dest_pkts',
                   14:'local'}, axis = 'columns')
    print(logs.shape, labelled_logs.shape)
    merged_logs = pd.concat([logs, labelled_logs],axis=1, sort=False)
    del logs
    del labelled_logs
    merged_logs["label"] = merged_logs.apply(f, axis=1)
    merged_logs = merged_logs[merged_logs.label == -1]
    rows = merged_logs.shape[0] + rows
    merged_logs.to_csv(output_dir + file + "unknown.csv", index = False)   
    stop = time.time()
    print("Complete. Time elapsed: "+ str(stop - start))

partition
(11063784, 13) (11063784, 4)
